# 🏛️ บทที่ 4.1: ถอดรหัสธุรกิจ (Domain Analysis)
---

จากไฟล์เคลมและไฟล์ชดเชยที่ป๋าส่งมา เรามามองข้าม "ตาราง Excel" แล้วมองเป็น "วัตถุในโลกจริง" (Real-world Objects) กันครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Domain Model** | โด-เมน โม-เดล | แบบจำลองของระบบธุรกิจที่เรากำลังจะเขียนโปรแกรม |
| **Value Object** | แวล-ลู ออบ-เจกต์ | ข้อมูลที่เป็นส่วนประกอบ (เช่น จำนวนเงิน, รหัสพัสดุ) ไม่มี ID ของตัวเอง |
| **Entity** | เอน-ติ-ตี | วัตถุหลักที่มีเอกลักษณ์ (ID) ชัดเจน (เช่น ใบเคลม, ใบชดเชยเงิน) |

**🔍 สิ่งที่เราสกัดได้จากไฟล์ของป๋า:**
1. **TrackingNumber (รหัสพัสดุ):** เป็น Value Object ที่ห้ามเป็นค่าว่าง และต้องนำไปจับคู่ได้
2. **Money (จำนวนเงิน):** เป็น Value Object ที่ประกอบด้วย ตัวเลข (Amount) และ สกุลเงิน (Currency)
3. **ClaimTicket (ใบเคลม):** เป็น Entity หลักที่มี `TrackingNumber` เป็นส่วนประกอบ

# 🔴 บทที่ 4.2: กฎข้อแรกของ TDD (Red Phase)
---

เราจะเริ่มจากสิ่งที่เล็กที่สุดคือ **TrackingNumber (รหัสพัสดุ)** ครับ 
ในทางวิศวกรรม เราจะไม่ยอมให้โปรแกรมรับ String มั่วๆ (เช่น `"123"`, `""`, หรือ `None`) เข้ามาในระบบ เราจึงต้องสร้าง Value Object มาคลุมมันไว้ และใช้ `Pydantic` ในการตรวจสอบ (Validate) ความถูกต้องครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Red Phase** | เรด เฟส | ขั้นตอนแรกของ TDD คือการเขียน Test ให้ "พัง" (Fail) ก่อน เพราะเรายังไม่ได้เขียนโค้ดจริง |
| **Validation** | แว-ลิ-เด-ชัน | การตรวจสอบความถูกต้องของข้อมูล |

In [3]:
import pytest
from pydantic import ValidationError

# สมมติว่าเราเรียกใช้คลาสจากไฟล์ domain (ที่เรากำลังจะสร้าง)
# from domain import TrackingNumber 

# -----------------------------------------
# Test Cases สำหรับ TrackingNumber
# -----------------------------------------

# เทสที่ 1: ใส่ข้อมูลถูกต้อง ต้องทำงานได้
def test_valid_tracking_number():
    # สมมติเราดึงรหัส TH1234567890 มาจากไฟล์ Excel ของป๋า
    tracking = TrackingNumber(value="TH1234567890")
    assert tracking.value == "TH1234567890"

# เทสที่ 2: ใส่ค่าว่าง ต้องพัง (Error)
def test_empty_tracking_number():
    with pytest.raises(ValidationError):
        TrackingNumber(value="") # ห้ามว่างเด็ดขาด

# เทสที่ 3: ใส่ประเภทผิด (เช่น ใส่ตัวเลข) ต้องพัง
def test_invalid_type_tracking_number():
    with pytest.raises(ValidationError):
        TrackingNumber(value=12345678) # ต้องเป็น String เท่านั้น

# 🟢 บทที่ 4.3: ทำให้ Test ผ่าน (Green Phase)
---

พอเรามี Test ดักไว้แล้ว ทีนี้เราจะมาเขียนโค้ด Domain จริงๆ กันครับ เราจะใช้ `Pydantic` ซึ่งเป็นตัวจัดการ Value Object ที่ดีที่สุดใน Python ณ ตอนนี้ครับ

In [ ]:
from pydantic import BaseModel, Field, field_validator

class TrackingNumber(BaseModel):
    """
    Value Object สำหรับจัดการรหัสพัสดุ (Tracking Number)
    """
    # กฎ: ต้องเป็น String, ห้ามว่าง (min_length=1), เว้นวรรคหน้าหลังจะถูกตัดทิ้ง (strip_whitespace)
    value: str = Field(..., min_length=1, json_schema_extra={"strip_whitespace": True})

    # เราสามารถเพิ่มกฎ Business Logic ย่อยๆ ได้ด้วย เช่น รหัสต้องยาวกว่า 3 ตัว
    @field_validator('value')
    def check_length(cls, v):
        if len(v) < 5:
            raise ValueError('Tracking Number สั้นเกินไป ไม่น่าจะใช่ของจริง')
        return v.strip()

# --- ลองใช้งานจริง ---
if __name__ == "__main__":
    # 1. ลองใส่รหัสที่ถูกต้อง (จำลองจากไฟล์ป๋า)
    good_track = TrackingNumber(value="TH1234567890")
    print(f"✅ สำเร็จ: ได้รหัสพัสดุ {good_track.value}")

    # 2. ลองแกล้งใส่รหัสสั้นๆ
    try:
        bad_track = TrackingNumber(value="TH1")
    except Exception as e:
        print(f"❌ โดนดักจับ Error: {e}")

✅ สำเร็จ: ได้รหัสพัสดุ TH1234567890
❌ โดนดักจับ Error: 1 validation error for TrackingNumber
value
  Value error, Tracking Number สั้นเกินไป ไม่น่าจะใช่ของจริง [type=value_error, input_value='TH1', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error
